In [1]:
from statsforecast.adapters.prophet import AutoARIMAProphet
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import warnings
warnings.filterwarnings("ignore")

2022-07-05 08:59:37 prophet.plot ERROR: Importing plotly failed. Interactive plots will not work.


In [2]:
data =  pd.read_csv('../../../Data/pv_power_df_5day_capacity_scaled.csv', index_col='datetime').drop(columns=['2657', '2828']) #DROPPING FAULTY SYSTEMS
uk_pv = pd.read_csv('../../../Data/system_metadata_location_rounded.csv')
uk_pv['ss_id_string'] = uk_pv['ss_id'].astype('str')

In [3]:
start = time.time()
n_sys = 50

models = dict.fromkeys(data.columns[:n_sys])
df_p_tot = dict.fromkeys(data.columns[:n_sys])
df_cv_tot = dict.fromkeys(data.columns[:n_sys])

n_test = 24
n_data = 5000

for idx_series, system_id in enumerate(data.columns[:n_sys]):
    print(f'constructing time series for system {system_id}')
    ts = data.iloc[:n_data, idx_series].to_frame()
    ts = ts.T.stack().to_frame().reset_index().set_index('level_0')
    ts = ts.rename(index={'level_0':'unique_id'}, columns = {'datetime':'ds', 0:'y'})
    ts.index = ts.index.rename('unique_id')

    #following this to put constant trend: https://github.com/facebook/prophet/issues/614
    print(f'defining the model')
    m = AutoARIMAProphet(growth = 'flat', yearly_seasonality = True, weekly_seasonality =False, daily_seasonality = True,
                            seasonality_mode = 'multiplicative', 
                            d=1,
                            D=1,
                            max_p=2,
                            max_q=2,
                            max_P=2,
                            max_Q=2,
                            start_p=1,
                            start_q=1,
                            start_P=1,
                            start_Q=1,
                            stationary=False,
                            seasonal=True,
                            ic='aicc',
                            stepwise=True,
                            nmodels=94,
                            test='kpss',
                            seasonal_test='seas',
                            num_cores=2,
                            period=97,
    )
    print('fit the model')
    m.fit(ts)

    print('create the future df')
    day_max = 16
    day_min = 8
    future = m.make_future_dataframe(periods=1000, freq = '5min')
    future2 = future[(future.ds.dt.hour >= 8)  & (future.ds.dt.hour < 16) ]
    future2 = future2.iloc[:len(ts) + n_test]

    print('forecast')
    forecast = m.predict(future2).dropna(subset='ds')
    
    print('perform CV')
    init = str(int((1000/97) / 2)) + ' days' #start midway trough the ts
    period = str(max(1, int((1000/97) / 2) / 50)) + ' days ,  2 hours' #divide into 50 predictions, add 2 hours
    df_cv = cross_validation(m, initial=init, period=period, horizon = '2 hours')
    df_p = performance_metrics(df_cv)

    print('save the data in a dict')
    models[system_id] = m.arima.model_
    df_p_tot[system_id] = df_p
    df_cv_tot[system_id] = df_cv
    
end =  time.time()
time_loop = end - start
print(f'time it took is {time_loop}')

accuracy_df = pd.concat(df_p_tot)
mean_accuracy_result = accuracy_df.droplevel(1).groupby(axis=0, level=0).mean() 
models_df = pd.DataFrame.from_dict(models, orient='index')

mean_accuracy_result.to_csv('accuracies')
models_df.to_csv('models')


constructing time series for system 2607
defining the model
fit the model


KeyboardInterrupt: 

In [ ]:
# pd.DataFrame.from_dict(df_p_tot, orient='index')
# df_cv_tot['2607'].set_index('ds') #[['yhat', 'y']].plot(marker='.')

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mae')

fig1 = m.plot(forecast,)
plt.xlim(forecast.ds[800], forecast.ds[1023])
m.arima.summary()